# SWMM Simulated Streamflow in Fall Creek, NY

The Fall Creek watershed is a 325 km<sup>2</sup> basin, located near Ithaca, New York. To complete Sobol, delta, and OLS sensitivity analyses on SWMM simulated streamflow to input parameters, we employed the Saltelli sampling scheme (described in more detail in Section 3) to create 24,000 unique parameter sets. The code is available in the "Sampling.py" script. Each parameter set was run through the Storm Water Management Model (SWMM) from January 1, 2013 through June 30, 2013, and the time series of simulated streamflow was extracted. Simulated streamflow was compared to USGS stream gauge data (waterdata.usgs.gov). Six objective functions were calculated and used as model performance metrics using the R “hydroGOF” package: mean absolute error (MAE), mean error (ME), mean squared error (MSE), Nash-Sutcliff efficiency (NSE), percent bias (pbias), and root mean squared error (RMSE).

This jupyter notebook has code cells to:
1. Load data for use in R and Python scripts
2. Calculate objective function values (i.e. performance metrics)
3. Create a historical time series and magnitude percentile plots for simulated data
4. Generate a priori and a posteriori parameter distribution plots via Approximate Bayesian Computation
5. Compute Sobol first-order, Sobol second-order, Sobol total-order, delta, and OLS sensitivity indices
6. Produce portrait, scatter, and spider plots

To complete and visualize a comprehensive sensitivity analysis on the SWMM simulations, we use packages from both R and Python. This command allows us to run R scripts in the Python jupyter notebook.

In [ ]:
%load_ext rpy2.ipython

To calculate the objective functions for the SWMM simulated streamflow, we use the [hydroGOF](https://cran.r-project.org/web/packages/hydroGOF/hydroGOF.pdf) package in R. For the SWMM streamflow simulations in the Fall Creek, NY watershed, we calculate mean error, mean absolute error, mean squared error, Nase-Sutcliffe efficiency, percent bias, and root mean squared error. The objective function evaluations are saved in the "input/" folder to be loaded into the sensitivity analysis.

In [ ]:
%%R 
# this rpy2 'Rmagic' command allows to run the entire cell block as R code

# load in the R script that calculates six objective functions on SWMM model simulations versus USGS streamgage data
# %load Calculate_Objective_Functions.R

library(dplyr)
library(hydroGOF)

# load in observation and simulation data
obs <- read.csv("input/observation_ts.csv", header = TRUE)
  # "time_steps" row, "index" and "value" columns 

sim <- read.csv("input/simulation_ts.csv", header = TRUE) %>%
    dplyr::select(-1)
  # "model_runs" rows, "time_steps" columns

model_runs <- nrow(sim)
time_steps <- ncol(sim)

mean_error <- array(NA, model_runs)
mean_abs_error <- array(NA, model_runs)
mean_sq_error <- array(NA, model_runs)
root_mse <- array(NA, model_runs)
p_bias <- array(NA, model_runs)
nse <- array(NA, model_runs)

for (i in 1:model_runs) {

  print(i)

  mean_error[i] <- me(sim = as.numeric(sim[i, 1:time_steps]), obs = as.numeric(obs[, 2]))
  mean_abs_error[i] <- mae(sim = as.numeric(sim[i, 1:time_steps]), obs = as.numeric(obs[, 2]))
  mean_sq_error[i] <- mse(sim = as.numeric(sim[i, 1:time_steps]), obs = as.numeric(obs[, 2]))
  root_mse[i] <- rmse(sim = as.numeric(sim[i, 1:time_steps]), obs = as.numeric(obs[, 2]))
  p_bias[i] <- pbias(sim = as.numeric(sim[i, 1:time_steps]), obs = as.numeric(obs[, 2]))
  nse[i] <- NSE(sim = as.numeric(sim[i, 1:time_steps]), obs = as.numeric(obs[, 2]))

}

OF <- as.data.frame(mean_error) %>%
  setNames("me") %>%
  dplyr:: mutate(mae = mean_abs_error,
                 mse = mean_sq_error,
                 rmse = root_mse,
                 pbias = p_bias,
                 nse = nse)

write.table(OF, "/input/OF_values.txt", sep = " ",
            row.names = FALSE, col.names = FALSE)

Now we load the model simulation data, observation/truth data, parameter sets, time stamps, and objective function values into Python as data frames using the pandas library.

In [ ]:
import pandas as pd

sim = pd.read_csv("input/simulation_ts.csv", index_col = 0)
obs = pd.read_csv("input/observation_ts.csv")
pars = pd.read_csv("input/params.csv", header = 0)
pars.columns = ['w', 'n_imperv', 'n_perv', 's_imperv', 's_perv', 'k_sat', 'per_routed', 'cmelt', 'Tb', 'A1', 'B1']
timestamps = pd.read_csv("input/timestamps.csv")
OF = pd.read_csv("input/OF_values.csv")

### 1. Graph Observed and Modeled Output

The following code produces time series plots for the observation/truth data. Since we have several thousand model simulation time series, visualizing them on a typical time series figure does not convey any significant meaning. Instead, we can rank the observations to get an idea of how often we exceed certain levels and create [magnitude percentile plots](https://waterprogramming.wordpress.com/2019/02/26/magnitude-varying-sensitivity-analysis-and-visualization-part-1/). The output plots saves to the "output/plots/magnitude_perc" directory.

In [ ]:
# %load Magnitude_Perc.py
#!/usr/bin/env python3

import pandas as pd
import os

# back out a directory to load python functions from "Scripts" folder
org_dir_name = os.path.dirname(os.path.realpath('Magnitude_Perc.py'))
parent_dir_name = os.path.dirname(os.path.dirname(os.path.realpath('Magnitude_Perc.py')))
os.chdir(parent_dir_name + "/Scripts")

# load python functions from ‘Scripts’ folder
import magnitude_percentile_plots

# move back into case study 0 folder
os.chdir(org_dir_name)

# load in model simulationsand observation data
sim = pd.read_csv("input/simulation_ts.csv", index_col = 0)
obs = pd.read_csv("input/observation_ts.csv")

# create magnitude percentile plots from observation and simulation time series data
magnitude_percentile_plots.magnitude_perc_plots(sim, obs)

### 2. Approximate Bayesian Computation
Approximate Bayesian computation (ABC) represents the combination of model parameter values that maximize the probability of representing the observed data. ABC bypasses the evaluation of the likelihood function by approximating the likelihood function by using simulations compared to the observed data. For more information on ABC, see Engeland and Gottschalk (2002), Neuman (2003), Sunnaker et al. (2013) etc. 

The steps and requirements for approximate Bayesian calculation are: 1) Observed data has a known mean and standard deviation and the user defines a summary statistic (i.e. objective function) 2) Assume you don’t know anything about the parameters, so assume a uniform prior interval [0,1]. 3) A total of n parameters are drawn from prior, the model is simulated for each of the parameter points , which results in n sequences of simulated data. 4) Calculate the summary statistic for each sequence of simulated data 5) Calculate distance between observed and simulated transition frequencies for all parameter points. Specify some tolerance  and “keep” parameter points smaller than or equal to the summary statistic as approximate samples from the posterior.

For the python script below, specify the number of model runs, tolerance of the objective functions, number of histogram bins and the figure colors. Here we used pre-defined objective functions, but the code can be modified to calculate a variety of objective functions. The plots produced are histograms of the various parameters illustrating the difference between original modeled output and the ABC constrained parameter sets.

In [ ]:
# %load approx_bayes_calc_of_defined.py 
#!/usr/bin/env python3

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

'''
Approximate Baysian Calculation requires:
    1) observation dataset (df_obs)
    2) parameter sets (df_parms)
    3) model output (df_model)
    4) objective functions (df_OFs)
    5) tolerance
    6) number of model runs
'''

def approx_bayes_calc_OF(parms,OFs,simulations):
    keep_nse = []; keep_pbias = []; keep_rmse = []
    for i in np.arange((simulations)):
        # User can redefine tolerance and OF here
        if tolerance_rmse < OFs.iloc[i,3]:
            keep_rmse.append(parms.loc[i])
            
        if np.absolute(OFs.iloc[i,4]) < tolerance_pbias:
            keep_pbias.append(parms.loc[i])
            
        if OFs.iloc[i,5] >= tolerance_nse:
            keep_nse.append(parms.loc[i])        
    return keep_nse,keep_pbias,keep_rmse

plt.rcParams.update({'font.size': 18})
def make_histograms(df_parms,bayes_approx,bins,alpha,cc1,cc2,parameters,metric):
    plt.figure(figsize=(12,12))
    for col in np.arange(1,((df_parms.iloc[0,:]).size)+1):
        plt.subplot(4,3,col)
        ax = df_parms.iloc[:,col-1].plot.hist(bins=bins,alpha=alpha,color=cc1)    
        ax = bayes_approx.iloc[:,col-1].plot.hist(bins=bins,alpha=alpha,color=cc2)
        ax.set_xlabel(str(parameters[col-1]))    
    plt.legend(['Output','ABC'],fancybox=True)
    plt.tight_layout() 
    plt.savefig('output/'+metric+'.png',dpi=300)

def make_cdfs_pdfs(df_parms,bayes_approx,bins,alpha,cc1,cc2,parameters,metric):
    plt.figure(figsize=(12,12))
    for col in np.arange(1,((df_parms.iloc[0,:]).size)):
        plt.subplot(4,3,col)
        ax = df_parms.iloc[:,col].plot.hist(cumulative=True, density=1,bins=bins,alpha=alpha,color=cc1)    
        ax = bayes_approx.iloc[:,col].plot.hist(cumulative=True, density=1,bins=bins,alpha=alpha,color=cc2)
        ax.set_xlabel(str(parameters[col]))    
    plt.legend(['Output','ABC'],fancybox=True)
    plt.tight_layout() 
    plt.savefig('output/'+metric+'_cdf.png',dpi=300)
    
    plt.figure(figsize=(12,12))
    for col in np.arange(1,((df_parms.iloc[0,:]).size)):
        plt.subplot(4,3,col)
        ax = df_parms.iloc[:,col].plot.kde(alpha=alpha,color=cc1)    
        ax = bayes_approx.iloc[:,col].plot.kde(alpha=alpha,color=cc2)
        ax.set_xlabel(str(parameters[col]))    
    plt.legend(['Output','ABC'],fancybox=True)
    plt.tight_layout() 
    plt.savefig('output/'+metric+'_pdf.png',dpi=300)

def runABC(df_parms,df_OFs,runs,bins,color1,color2):
    # models with objective functions within tolerance thresholds
    results_nse,results_pbias,results_rmse = np.array(approx_bayes_calc_OF(df_parms,df_OFs,runs))
    
    # saves models with objective functions within tolerance thresholds
    bayes_approx_nse = pd.DataFrame(results_nse,columns=None)
    bayes_approx_nse.to_csv('output/bayes_parameters_NSE.csv',index=False)
    bayes_approx_pbias = pd.DataFrame(results_pbias,columns=None)
    bayes_approx_pbias.to_csv('output/bayes_parameters_pbias.csv',index=False)
    bayes_approx_rmse = pd.DataFrame(results_rmse,columns=None)
    bayes_approx_rmse.to_csv('output/bayes_parameters_rmse.csv',index=False)
    
    parameters = list(df_parms.columns.values)
    
    # print ABC results and make figures
    print('precent of models with NSE >= to',str(tolerance_nse),'are:',str(len(results_nse)/runs),'%')
    make_histograms(df_parms,bayes_approx_nse,bins,0.5,color1,color2,parameters,'NSE')
    make_cdfs_pdfs(df_parms,bayes_approx_nse,bins,0.5,color1,color2,parameters,'NSE')
    
    print('precent of models with',str(-tolerance_pbias),'<= p-bias >=',str(tolerance_pbias),'are:',str(len(results_pbias)/runs),'%')
    make_histograms(df_parms,bayes_approx_pbias,bins,0.5,color1,color3,parameters,'pbias')
    make_cdfs_pdfs(df_parms,bayes_approx_nse,bins,0.5,color1,color3,parameters,'pbias')
    
    print('precent of models with RMSE < to',str(tolerance_rmse),'are:',str(len(results_rmse)/runs),'%')
    make_histograms(df_parms,bayes_approx_rmse,bins,0.5,color1,color4,parameters,'RMSE')
    make_cdfs_pdfs(df_parms,bayes_approx_nse,bins,0.5,color1,color4,parameters,'RMSE')


In [ ]:
# Specify tolerance for objective functions (OF)
tolerance_rmse = 6.0   # OF < tolerance
tolerance_pbias = 15.0 # -tolerance < OF > tolerance
tolerance_nse = 0.0  # OF >= tolerance

runs = 24000 # specify number of model runs
bins = 100   # specify number of histogram bins
color1 = 'b' # color of original model output
color2 = 'k' # color of 1st ABC applied to OF (NSE)
color3 = 'r' # color of 2nd ABC applied to OF (p-bias)
color4 = 'g' # color of 3rd ABC applied to OF (RMSE)

# Runs function that evaluates models outputs with approximate Bayesian calculations
runABC(pars, OF, runs, bins, color1, color2)

### 2. Sensitivity Analysis

Three sensitivity analyses are incorporated into the workflow: a variance-based sensitivity analysis, a moment-independent sensitivity analysis, and an ordinary least squares regression. The Sobol method ([Sobol, 2001](https://doi.org/10.1016/S0378-4754(00)00270-6)) is a variance-based global sensitivity analysis that yields first-order, second-order, and total-order sensitivity indices. Sobol’s method can effectively handle nonlinear responses and measures the effects of interactions within non-additive systems. It decomposes the variance of the model output into fractions which can be attributed to inputs or sets of inputs. The first-order sensitivity index (i.e. main effect index) quantifies parameter impact on model output variance by averaging over the variations in other input parameters. The second-order sensitivity index decomposes model variance by parameter interactions with one another. The total-order sensitivity index (i.e. total effect index) measures the contribution each parameter on model output across the first-order index and all higher-order indices. Our workflow employs the Saltelli scheme ([Saltelli, 2002](https://doi.org/10.1016/S0010-4655(02)00280-1); [Saltelli et al., 2010](https://doi.org/10.1016/j.cpc.2009.09.018)), which allows for the calculation of the first-order, second-order, and total-order sensitivity indices with fewer model runs than the traditional approach. However, since the calculation of Sobol second-order indices requires N * (2K + 2) model runs (where N is preset model runs and K is number of parameters), some models may be too computationally expensive to run with multiple input parameters ([Saltelli, 2002](https://doi.org/10.1016/S0010-4655(02)00280-1)). Therefore, we also include two additional sensitivity analyses which have no minimum number of model runs.

The delta index ([Borgonovo, 2007](https://doi.org/10.1016/j.ress.2006.04.015); [Borgonovo et al., 2012](https://www.sciencedirect.com/science/article/pii/S1364815211001617?via%3Dihub); [Plischke et al., 2013](https://doi.org/10.1016/j.ejor.2012.11.047)) is a moment-independent global sensitivity analysis. While less robust than indices returned by a variance-based sensitivity analysis, a moment-independent sensitivity analysis is a popular technique due to its computational efficiency and insensitivity to dependent parameters [19]. The delta sensitivity analysis searches for parameters with the greatest impact on the probability density function of model output. Delta indices capture non-linear and non-monotonic parameter-output dynamics.

Lastly, the ordinary least squares (OLS) regression yields a R2 coefficient, which quantifies the linear effects of model input parameters on model output variance. OLS regressions have long been employed throughout model sensitivity analyses and assume an explicit interaction between model output and any given parameter [Kleijnen, 1995](https://doi.org/10.1002/sdr.4260110403); [Pannell, 1997](https://doi.org/10.1016/S0169-5150(96)01217-0); [Zobitz et al., 2006](https://doi.org/10.1016/j.agrformet.2006.01.003).

To visualize objective function sensitivity to model input parameters, the following code produces radial convergence plots, scatter plots, portrait plots, and spider plots based on the outputs from the sensitivity analyses.

The Sobol and delta sensitivity indices are calculated using a modified version of the Python sensitivity analysis library ([SALib](https://salib.readthedocs.io/en/latest/index.html)) and the OLS regression is calculated using [StatsModels](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html) library in Python.

In [ ]:
# %load SensIndices_RCPlots.py
#!/usr/bin/env python3

# import python libraries
import pandas as pd
import os

# back out a directory to load python functions from "Scripts" folder
org_dir_name = os.path.dirname(os.path.realpath('SensIndices_RCPlots.py'))
parent_dir_name = os.path.dirname(os.path.dirname(os.path.realpath('SensIndices_RCPlots.py')))
os.chdir(parent_dir_name + "/Scripts")

# load python functions from ‘Scripts’ folder
import delta
import sobol
import ols
import radial_conv_plots
import magnitude_percentile_plots

# move back into case study 0 folder
os.chdir(org_dir_name)

# Define the model inputs
problem = {
    'num_vars': 11,
    'names': ['w', 'n_imperv', 'n_perv', 's_imperv', 's_perv', 'k_sat', 'per_routed', 'cmelt', 'Tb', 'A1', 'B1'],
    'bounds': [[500, 1500], # meters
               [0.01, 0.2],
               [0.01, 0.2],
               [0, 10],
               [0, 10],
               [0.01, 10],
               [0, 100],
               [0, 4],
               [-3, 3],
               [0.0001, 0.01],
               [1, 3]]
}

# load in model parameter sets (Saltelli sampled) and objective function values
pars = pd.read_csv("input/params.csv", header = 0)
OF = pd.read_csv("input/OF_values.csv")

# save the parameter names
param_names = problem['names']

# calculate Sobol first-order, second-order, and total-order indices
results_SI = []
results_SI = sobol.objective_function_sobol(problem, OF)

# create radial convergence plots based on results_SI
radial_conv_plots.radial_conv_plots(problem, results_SI, OF)

# calculate delta indices and sobol first-order indices
results_delta = []
results_delta = delta.objective_function_delta(problem, pars, OF)

# calculate R^2 from OLS regression
results_R2 = []
results_R2 = ols.objective_function_OLS(OF, pars, param_names)

From the sensitivity analysis results (calculated and exported from python), we can create portrait plots, scatter plots, and spider plots for various objective functions and parameter values.

First, the data is loaded and formatted into a usable format and then exported to a .csv file. Then the script creates additional plots to help visualize and convey parameter sensitivity.

In [ ]:
%%R
# %load Portrait_Scatter_Spider.R

library(dplyr)

# load in objective function values and parameter sets
pars <- read.csv("input/params.csv", header = TRUE)
  # "model_runs" rows, "num_pars" columns

OF <- read.csv("input/OF_values.csv", header = TRUE)
  # "model_runs" rows, "num_OF" columns

# save names of objective functions and parameters
OF_names <- colnames(OF)
param_names <- colnames(pars)

# set variables of number of model runs, time steps, and number of parameters
model_runs <- nrow(sim)
time_steps <- ncol(sim)
num_pars <- ncol(pars)
num_OF <- ncol(OF)

# load in results from delta, sobol, and ols sensitivity analyses (calculated in python script)
source("../Scripts/python_to_r_results.R")
results_sobol <- python_to_r_results(data_type = "sobol", param_names, OF_names)
results_delta <- python_to_r_results(data_type = "delta", param_names, OF_names)
results_ols <- python_to_r_results(data_type = "ols", param_names, OF_names)

# save as csv files
lapply(results_sobol, function(x) write.table(data.frame(x), 'output/formatted_sobol.csv', append = T, sep = ',' ))
lapply(results_delta, function(x) write.table(data.frame(x), 'output/formatted_delta.csv', append = T, sep = ',' ))
lapply(results_ols, function(x) write.table(data.frame(x), 'output/formatted_ols.csv', append = T, sep = ',' ))

# scatter plots of objective functions versus parameter values
source("../Scripts/scatterplots.R")
for (i in 1:num_OF) {
  
  # subset by objective function, i
  objective_fun <- OF[, i]

  # create scatterplots of all parameters versus objective function, i
  par_OF_scatter(params = pars, objective_fun, OF_name = colnames(OF)[i])
  
}

# portrait plots of objective functions versus parameter values
source("../Scripts/portrait_plots.R")
portrait_plot(results_sobol)
portrait_plot(results_delta)
portrait_plot(results_ols)

# spiders plots of objective functions versus parameter values
source("../Scripts/spider_plots.R")
spiderplot(results_sobol)
spiderplot(results_delta)
spiderplot(results_ols)

### XX. Conclusion

We did awesome stuff and this is how we feel about it...

### XX. References
- Engeland, K., Gottschalk L. Bayesian estimation of parameters in regional hydrological model, Hydrol. Earth Sys. Sci., **2002**, *6*(5), 883-898. https://doi.org/10.5194/hess-6-883-2002
- Neuman, S. Maximum likelihood Bayesian averaging of uncertain model predictions, Stoch. Environ. Res. Risk Assess. **2003** *17*, 291. https://doi.org/10.1007/s00477-003-0151-7
- Sunnåker M., Busetto A.G., Numminen E., Corander J., Foll M., Dessimoz C. Approximate Bayesian Computation, PLoS Comput. Biol. **2013** *9*(1): e1002803. https://doi.org/10.1371/journal.pcbi.1002803
